In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

### Daniel's Data

In [2]:
import h5py

In [3]:
data = h5py.File('/Users/daiyiluo/Downloads/ms.mat')

<ipython-input-3-e98d58a5d81c>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data = h5py.File('/Users/daiyiluo/Downloads/ms.mat')


In [9]:
list(data.keys())

['#refs#', '#subsystem#', 'ms']

In [12]:
data['ms'].keys()

<KeysViewHDF5 ['C', 'Centroids', 'CorrProj', 'Experiment', 'FR', 'FR05Per', 'FR50Per', 'FR95Per', 'FiltTraces', 'Options', 'PeakToNoiseProj', 'RawTraces', 'S', 'SFPs', 'analysis_time', 'b', 'binCenters', 'binSize', 'bruteForceTraces', 'camNumber', 'dirName', 'ds', 'frameNum', 'height', 'info', 'infoP', 'maxBufferUsed', 'maxFramesPerFile', 'meanFrame', 'numFiles', 'numFrames', 'numNeurons', 'occEven', 'occOdd', 'occThresh', 'pos', 'shifts', 'speed', 'speedThresh', 'stabilityCoef', 'stabilityCoef2', 'stabilityCoef2P', 'stabilityCoefP', 'time', 'trackLength', 'trialNum', 'vidNum', 'vidObj', 'vidObjMotionCorr', 'width']>

In [7]:
s=data['ms']['S']
type(s)

h5py._hl.dataset.Dataset

In [13]:
np.savez(f'/Users/daiyiluo/Downloads/caiman_t/dan_CA1/S.npz', spd=data['ms']['speed'], dis=data['ms']['pos'])

In [4]:
np.savez(f'/Users/daiyiluo/Downloads/caiman_t/dan_CA1/spk.npz', spks=data['ms']['S'].T, traces=data['ms']['C'].T)

AttributeError: 'Dataset' object has no attribute 'T'

### Fish's Data

### a. Load data

In [3]:
#----- find shape of trace data -----#

folder = '/Users/daiyiluo/Downloads/KQ086_210112' #KQ095_210205' #
data = loadmat(f'{folder}/dFishQ_raw.mat')
data.keys() # check the name of the variable

dict_keys(['__header__', '__version__', '__globals__', 'dFishQ_raw'])

In [4]:
key = 'dFishQ_raw' # 'B' for CA1 data, 'dFishQ_raw' for CA3 data
traces = data[key]
del data
print(f'Shape of calcium traces: {traces.shape}')

Shape of calcium traces: (28187, 99)


In [4]:
#----- load behaviour data -----#

data = loadmat(f'{folder}/S.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'S'])

In [5]:
key = 'S'
data[key].dtype

dtype([('runSpeed', 'O'), ('distance', 'O'), ('lick', 'O'), ('valve', 'O'), ('frameClock', 'O'), ('lapReset', 'O')])

In [6]:
# Locate the time point of each frame
FrameClock = np.round(data[key][0,0]['frameClock'])
d = np.diff(np.squeeze(FrameClock>4.5).astype(int))
idx = np.where(d<-0.5)[0]

print(f'Check if this equals to 2. \n #clock - #frames: {idx.shape[0]-traces.shape[0]}')

Check if this equals to 2. 
 #clock - #frames: 2


In [7]:
distance = data[key][0,0]['distance'].squeeze()
speed = data[key][0,0]['runSpeed'].squeeze()
lick = data[key][0,0]['lick'].squeeze()

del data

distance.shape, speed.shape, lick.shape, traces.shape

((9497531,), (9497531,), (9497531,), (28187, 99))

### b. Downsample behaviour data to frame rate

In [8]:
val, counts = np.unique(np.diff(idx), return_counts=True)
print(f'Frame interval: \n{val[counts>5]}\nOccurrence:     \n{counts[counts>5]}')
print('\nThen decide the frameinterv var below...')

Frame interval: 
[333 334]
Occurrence:     
[20194  7993]

Then decide the frameinterv var below...


In [9]:
frameinterv = 333  # most frame interval is 333 or 334
lk0 = np.convolve(lick, np.ones(frameinterv), 'valid') / frameinterv 
halflen = np.floor(frameinterv/2).astype('int16')
lk0 = np.concatenate((np.zeros(halflen),lk0, np.zeros(halflen)))
lk = lk0[idx[:-2]]

dis = distance[idx[:-2]]
spd = speed[idx[:-2]]

spd.shape,dis.shape,lk.shape

((28187,), (28187,), (28187,))

### c. Check time point of each frame

In [10]:
#-------- Check every thing ---------#
# Same lap length for every lap?
%matplotlib
plt.figure(figsize=(20,5))
plt.plot(FrameClock)
plt.plot(distance)
plt.plot(speed)
plt.plot(idx, FrameClock[idx], '*')
plt.show()

Using matplotlib backend: MacOSX


In [12]:
#-------- Check licking rate downsampleing --------#
plt.figure()
plt.plot(lick)
plt.plot(idx[:-2],lk,'*')
plt.plot(lk0)

In [11]:
#-------- Check speed downsampleing --------#
plt.plot(speed)
plt.plot(idx[:-2], spd)
# plt.xlim(4e6, 4.5e6)
plt.ylabel('speed')

Text(0, 0.5, 'speed')

### d. Save data with same sampling rate

In [12]:
spd.shape,dis.shape,lk.shape

((28187,), (28187,), (28187,))

In [13]:
# check if you need to truncate the data
st = 0
end = len(lk)
np.savez(f'{folder}/S.npz', spd=spd[st:end], dis=dis[st:end], lk=lk[st:end])

### e. Deconvolution for spike data

In [2]:
import warnings
warnings.filterwarnings('ignore')
from caiman.source_extraction import cnmf

In [5]:
traces.shape

(28187, 99)

In [6]:
spks = np.zeros_like(traces)
for i in range(traces.shape[1]):
    _, _, _, _, _, spks[:,i], _ = cnmf.deconvolution.constrained_foopsi(traces[:,i],p=2)

np.savez(f'{folder}/spk.npz', spks=spks)

print(f'Shape of deconvolved spikes: {spks.shape}')

Shape of deconvolved spikes: (28187, 99)


In [7]:
#-------- Check deconvolution --------#
%matplotlib
plt.figure()
plt.plot(traces[:,20])
plt.plot(spks[:,20])

Using matplotlib backend: MacOSX
